In [ ]:
import zipfile
import os

# Path to the uploaded .zip file
zip_path = '/kaggle/input/your-dataset-name/your-file.zip'  # Replace with your actual file path
extract_path = '/kaggle/working/dataset-folder'  # The path where you want to extract the files

# Extract the .zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List extracted files
print(os.listdir(extract_path))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight
from torchvision import datasets
from sklearn.metrics import accuracy_score
import os

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Paths to dataset (update for each dataset)
train_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/train"
valid_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/valid"
test_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/test"
save_path = "/kaggle/working/best_model_idrid.pth"

# Hyperparameters
num_epochs = 50
patience = 5
batch_size = 32
learning_rate = 0.0001

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Compute class weights
labels = [label for _, label in train_dataset.samples]
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model setup
model = models.mobilenet_v2(weights='IMAGENET1K_V1')
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers except classifier

# Modify classifier for binary classification
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.last_channel, 512),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512),
    nn.Dropout(0.3),
    nn.Linear(512, 1)  # Binary output (logit)
)
for param in model.classifier.parameters():
    param.requires_grad = True
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1, verbose=True)

# Training loop
best_val_loss = np.inf
epochs_without_improvement = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    valid_loss = 0.0
    valid_labels = []
    valid_preds = []

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            valid_loss += loss.item()

            preds = torch.sigmoid(outputs) > 0.5
            valid_labels.extend(labels.cpu().numpy())
            valid_preds.extend(preds.cpu().numpy())

    avg_valid_loss = valid_loss / len(valid_loader)
    val_accuracy = accuracy_score(valid_labels, valid_preds)
    print(f"Validation Loss: {avg_valid_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}")

    # Adjust learning rate
    scheduler.step(avg_valid_loss)

    # Save the best model
    if avg_valid_loss < best_val_loss:
        best_val_loss = avg_valid_loss
        epochs_without_improvement = 0
        best_model_wts = model.state_dict()
    else:
        epochs_without_improvement += 1

    # Early stopping
    if epochs_without_improvement >= patience:
        print("Early stopping triggered.")
        break

# Load best weights
model.load_state_dict(best_model_wts)

# Save the best model
torch.save(model.state_dict(), save_path)
print(f"Best model saved as '{save_path}'")

# Test the model
model.eval()
test_labels = []
test_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()
        outputs = model(inputs).squeeze()

        preds = torch.sigmoid(outputs) > 0.5
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

test_accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
train_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/train"
valid_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/valid"
test_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/test"

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets and DataLoaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class weights for imbalanced datasets
labels = [label for _, label in train_dataset.samples]
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Model setup
model = models.mobilenet_v2(weights='IMAGENET1K_V1')

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Update classifier layer
num_classes = 2
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.last_channel, 256),
    nn.ReLU(inplace=True),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

# Unfreeze classifier parameters
for param in model.classifier.parameters():
    param.requires_grad = True

# Move model to device
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1, verbose=True)

# Early stopping setup
patience = 5
best_val_loss = np.inf
epochs_without_improvement = 0
num_epochs = 50

# Training loop
train_losses = []
valid_losses = []
learning_rates = []

print("Starting training...\n")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    running_valid_loss = 0.0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_valid_loss += loss.item()

    avg_valid_loss = running_valid_loss / len(valid_loader)
    valid_losses.append(avg_valid_loss)

    # Learning rate scheduler step
    scheduler.step(avg_valid_loss)
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)

    # Epoch summary
    print(f"Epoch [{epoch+1}/{num_epochs}] Summary:")
    print(f"  Training Loss: {avg_train_loss:.4f}")
    print(f"  Validation Loss: {avg_valid_loss:.4f}")
    print(f"  Learning Rate: {current_lr:.6f}")

    # Early stopping
    if avg_valid_loss < best_val_loss:
        best_val_loss = avg_valid_loss
        torch.save(model.state_dict(), '/kaggle/working/best_model.pth')
        print("  Model improved. Saving model.\n")
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("  Early stopping triggered. Ending training.")
            break

# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

# Plotting
plt.figure(figsize=(10, 5))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses)+1), train_losses, label="Training Loss")
plt.plot(range(1, len(valid_losses)+1), valid_losses, label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')

# Plot learning rates
plt.subplot(1, 2, 2)
plt.plot(range(1, len(learning_rates)+1), learning_rates, label="Learning Rate")
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.legend()
plt.title('Learning Rate Over Epochs')

plt.tight_layout()
plt.show()


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 109MB/s] 
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting training...

Epoch [1/50] Summary:
  Training Loss: 0.3158
  Validation Loss: 0.2233
  Learning Rate: 0.001000
  Model improved. Saving model.

Epoch [2/50] Summary:
  Training Loss: 0.1963
  Validation Loss: 0.2105
  Learning Rate: 0.001000
  Model improved. Saving model.

Epoch [3/50] Summary:
  Training Loss: 0.1607
  Validation Loss: 0.1772
  Learning Rate: 0.001000
  Model improved. Saving model.

Epoch [4/50] Summary:
  Training Loss: 0.1489
  Validation Loss: 0.1897
  Learning Rate: 0.001000
Epoch [5/50] Summary:
  Training Loss: 0.1496
  Validation Loss: 0.1544
  Learning Rate: 0.001000
  Model improved. Saving model.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
train_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/train"
valid_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/valid"
test_dir = r"/kaggle/input/d/fatimafarwah/fyp-dataset/idrid_segmented/test"

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets and DataLoaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class weights for imbalanced datasets
labels = [label for _, label in train_dataset.samples]
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Model setup
model = models.mobilenet_v2(weights='IMAGENET1K_V1')

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Update classifier layer
num_classes = 2
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.last_channel, 256),
    nn.ReLU(inplace=True),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

# Unfreeze classifier parameters
for param in model.classifier.parameters():
    param.requires_grad = True

# Move model to device
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1, verbose=True)

# Early stopping setup
patience = 5
best_val_loss = np.inf
epochs_without_improvement = 0
num_epochs = 50

# Training loop
train_losses = []
valid_losses = []
learning_rates = []

print("Starting training...\n")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    running_valid_loss = 0.0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_valid_loss += loss.item()

    avg_valid_loss = running_valid_loss / len(valid_loader)
    valid_losses.append(avg_valid_loss)

    # Learning rate scheduler step
    scheduler.step(avg_valid_loss)
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)

    # Epoch summary
    print(f"Epoch [{epoch+1}/{num_epochs}] Summary:")
    print(f"  Training Loss: {avg_train_loss:.4f}")
    print(f"  Validation Loss: {avg_valid_loss:.4f}")
    print(f"  Learning Rate: {current_lr:.6f}")

    # Early stopping
    if avg_valid_loss < best_val_loss:
        best_val_loss = avg_valid_loss
        torch.save(model.state_dict(), '/kaggle/working/best_model.pth')
        print("  Model improved. Saving model.\n")
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("  Early stopping triggered. Ending training.")
            break

# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

# Plotting
plt.figure(figsize=(10, 5))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses)+1), train_losses, label="Training Loss")
plt.plot(range(1, len(valid_losses)+1), valid_losses, label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')

# Plot learning rates
plt.subplot(1, 2, 2)
plt.plot(range(1, len(learning_rates)+1), learning_rates, label="Learning Rate")
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.legend()
plt.title('Learning Rate Over Epochs')

plt.tight_layout()
plt.show()


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting training...

Epoch [1/50] Summary:
  Training Loss: 0.2773
  Validation Loss: 0.1998
  Learning Rate: 0.001000
  Model improved. Saving model.

